In [5]:
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib widget

In [83]:
file_ha = np.genfromtxt('hatp11/HAT-P-11_Halpha_PyORBIT.dat')
file_rv = np.genfromtxt('hatp11/HAT-P-11_RVs_PyORBIT.dat')
file_rj = np.genfromtxt('hatp11/HAT-P-11_RVs_PyORBIT_JitterFlag.dat')
file_si = np.genfromtxt('hatp11/HAT-P-11_Sindex_PyORBIT.dat')


rv_model = np.genfromtxt('hatp11/RV_data_radial_velocities_b.dat', names=True)


time = []
values = []
errors = []

time.append(file_rv[:,0])
values.append(file_rv[:,1]-rv_model['val_compare'] + 2.4)
errors.append(file_rv[:,2])

time.append(file_si[:,0])
values.append(file_si[:,1]- 0.5729)
errors.append(file_si[:,2])

time.append(file_ha[:,0])
values.append(file_ha[:,1]-0.045472)
errors.append(file_ha[:,2])



dataset_x0 = []
dataset_res = []
dataset_label = []
dataset_er2 = []
dataset_ej2 = []
#for ii in range(0, 3):
#    temp_input = np.append(temp_input, tsmooth)
#    temp_label = np.append(temp_label, np.zeros_like(tsmooth, dtype=int) + ii)
#
#X_input = (temp_input, temp_label.astype(int))


dataset_x0 = np.append(dataset_x0, file_rv[:,0])
dataset_label = np.append(dataset_label, np.zeros_like(file_rv[:,0], dtype=int) + 0)
dataset_res = np.append(dataset_res, file_rv[:,1]-rv_model['val_compare'] - -2.4)
dataset_er2 = np.append(dataset_er2, file_rv[:,2]**2)
dataset_ej2 = np.append(dataset_ej2, file_rv[:,2]**2 + 4.59**2)

dataset_x0 = np.append(dataset_x0, file_si[:,0])
dataset_label = np.append(dataset_label, np.zeros_like(file_si[:,0], dtype=int) + 1)
dataset_res = np.append(dataset_res, file_si[:,1]-0.5729)
dataset_er2 = np.append(dataset_er2, file_si[:,2]**2)
dataset_ej2 = np.append(dataset_ej2, file_si[:,2]**2 + 0.0021**2)

dataset_x0 = np.append(dataset_x0, file_ha[:,0])
dataset_label = np.append(dataset_label, np.zeros_like(file_ha[:,0], dtype=int) + 2)
dataset_res = np.append(dataset_res, file_ha[:,1]-0.045472)
dataset_er2 = np.append(dataset_er2, file_ha[:,2]**2)
dataset_ej2 = np.append(dataset_ej2, file_ha[:,2]**2 + 0.000328**2)



In [84]:
from spleaf import cov, term

t_full, y_full, yerr_full, series_index = cov.merge_series(time, values, errors)


In [66]:
yerr_jit = yerr_full.copy()
yerr_jit[series_index[0]] = np.sqrt(file_rv[:,2]**2 + 4.59**2)
yerr_jit[series_index[1]] = np.sqrt(file_si[:,2]**2 + 0.0021**2)
yerr_jit[series_index[2]] = np.sqrt(file_ha[:,2]**2 + 0.000328**2)

In [67]:
D = cov.Cov(t_full,
  err=term.Error(yerr_jit),
  GP=term.MultiSeriesKernel(term.ESPKernel(1.0,  28.93, 64.3, 1.857, nharm=4), series_index,
  [-12.4, 0.0004, -0.000009], [45.9,  0.927, 0.00404 ]))
D_loglike = D.loglike(y_full)

print(D_loglike)


input_param = [28.93, 64.3, 1.857, -12.4, 0.0004, -0.000009, 45.9,  0.927, 0.00404]

D_param = D.param[1:]
D.set_param(input_param, D_param)
D_loglike = D.loglike(y_full)
print(D_loglike)

D_param = D.param[1:]
D.set_param(input_param, D_param)
D_loglike = D.loglike(y_full)
print(D_loglike)

882.4737849788542
882.4737849788542
882.4737849788542


In [85]:
kwargs = {
    'err': term.Error(yerr_full),
    'GP': term.MultiSeriesKernel(term.ESPKernel(1.0, 2., 1000.0, 0.35, nharm=4), series_index,
  np.ones(3), np.zeros(3)),
    'jitter_inst00': term.InstrumentJitter(series_index[0], 1.0000000000000000),
    'jitter_inst01': term.InstrumentJitter(series_index[1], 1.0000000000000000),
    'jitter_inst02': term.InstrumentJitter(series_index[2], 1.0000000000000000),
}

input_param = [28.93, 64.3, 1.857, -12.4, 0.0004, -0.000009,45.9,  0.927, 0.00404, 4.59, 0.0021, 0.000328]


D1 = cov.Cov(t_full, **kwargs)
D1_param = D1.param[1:]
D1.set_param(input_param, D1_param)

D1_loglike = D1.loglike(y_full)

print(D1_loglike)


754.583820849766


In [86]:
kwargs = {
    'err': term.Error(yerr_full),
    'GP': term.MultiSeriesKernel(term.ESPKernel(1.0, 28.93, 64.3, 1.857, nharm=4), series_index,
   [-12.4, 0.0004, -0.000009], [45.9,  0.927, 0.00404 ]),
    'jitter_inst00': term.InstrumentJitter(series_index[0], 4.59),
    'jitter_inst01': term.InstrumentJitter(series_index[1], 0.0021),
    'jitter_inst02': term.InstrumentJitter(series_index[2], 0.000328),
}
kwargs2 = kwargs.copy()

input_param = [28.93, 64.3, 1.857, -12.4, 0.0004, -0.000009,45.9,  0.927, 0.00404, 4.59, 0.0021, 0.000328]

D2 = cov.Cov(t_full, **kwargs)

D2_loglike = D2.loglike(y_full)


print(D2_loglike)
D2_param = D2.param[1:]
D2.set_param(input_param, D2_param)
D2_loglike = D2.loglike(y_full)
print(D2_loglike)

input_param = [28.93, 64.3, 1.857, -12.4, 0.0004, -0.000009,45.9,  0.927, 0.00404, 0, 0.0, 0.00]

#input_param = [28.93, 64.3, 1.857, -12.4, 0.0004, -0.000009,45.9,  0.927, 0.00404, 4.59, 0.0021, 0.000328]

D2.set_param(np.asarray(input_param), D2_param)
D2_loglike = D2.loglike(y_full)
print(D2_loglike)
D2.set_param(input_param, D2_param)
D2_loglike = D2.loglike(y_full)
print(D2_loglike)


882.4737849788542
882.4737849788542
965.0335529481865
965.0335529481865


In [87]:
import sys
import jax
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp
from tinygp import kernels, GaussianProcess
#from tinygp.helpers import JAXArray

if sys.version_info[1] < 10:
    raise Warning("You should be using Python 3.10 - tinygp may not work")

class LatentKernel(kernels.Kernel):
    """A custom kernel based on Rajpaul et al. (2015)

    Args:
        kernel: The kernel function describing the latent process. This can be any other
            ``tinygp`` kernel.
        coeff_prim: The primal coefficients for each class. This can be thought of as how
            much the latent process itself projects into the observations for that class.
            This should be an array with an entry for each class of observation.
        coeff_deriv: The derivative coefficients for each class. This should have the same
            shape as ``coeff_prim``.
    """

    try:
        kernel : kernels.Kernel
        coeff_prim: jax.Array | float
        coeff_deriv: jax.Array | float
    except:
        pass

    def __init__(self, kernel, coeff_prim, coeff_deriv):
        self.kernel = kernel
        self.coeff_prim, self.coeff_deriv = jnp.broadcast_arrays(
            jnp.asarray(coeff_prim), jnp.asarray(coeff_deriv)
        )

    def evaluate(self, X1, X2):
        t1, label1 = X1
        t2, label2 = X2

        # Differentiate the kernel function: the first derivative wrt x1
        Kp = jax.grad(self.kernel.evaluate, argnums=0)

        # ... and the second derivative
        Kpp = jax.grad(Kp, argnums=1)

        # Evaluate the kernel matrix and all of its relevant derivatives
        K = self.kernel.evaluate(t1, t2)
        d2K_dx1dx2 = Kpp(t1, t2)

        # For stationary kernels, these are related just by a minus sign, but we'll
        # evaluate them both separately for generality's sake
        dK_dx2 = jax.grad(self.kernel.evaluate, argnums=1)(t1, t2)
        dK_dx1 = Kp(t1, t2)

        # Extract the coefficients
        a1 = self.coeff_prim[label1]
        a2 = self.coeff_prim[label2]
        b1 = self.coeff_deriv[label1]
        b2 = self.coeff_deriv[label2]

        # Construct the matrix element
        return (
            a1 * a2 * K
            + a1 * b2 * dK_dx2
            + b1 * a2 * dK_dx1
            + b1 * b2 * d2K_dx1dx2
        )


def _build_tinygp_multidimensional(params):

    base_kernel = kernels.ExpSquared(scale=jnp.abs(params["Pdec"])) \
            * kernels.ExpSineSquared(
            scale=jnp.abs(params["Prot"]),
            gamma=jnp.abs(params["gamma"]))

    kernel = LatentKernel(base_kernel, params['coeff_prime'], params['coeff_deriv'])
    return GaussianProcess(
        kernel, params['X'], diag=jnp.abs(params['diag']), mean=0.0
    )

@jax.jit
def _loss_tinygp(params):
    gp = _build_tinygp_multidimensional(params)
    return gp.log_probability(params['y'])

In [88]:

tinygp_X = (dataset_x0, dataset_label.astype(int))

input_param = [28.93, 64.3, 1.857, -12.4, 0.0004, -0.000009,45.9,  0.927, 0.00404, 4.59, 0.0021, 0.000328]
theta_dict =  dict(
    gamma=1. / (2.*input_param[2] ** 2),
    Pdec=input_param[1],
    Prot=input_param[0],
    diag=dataset_ej2,
    X=tinygp_X,
    y=dataset_res,
    coeff_prime=input_param[3:6],
    coeff_deriv=input_param[6:9],
    #x_predict = X_input
)
tinygp_log =  _loss_tinygp(theta_dict)
print(tinygp_log)


theta_dict =  dict(
    gamma=1. / (2.*input_param[2] ** 2),
    Pdec=input_param[1],
    Prot=input_param[0],
    diag=dataset_er2,
    X=tinygp_X,
    y=dataset_res,
    coeff_prime=input_param[3:6],
    coeff_deriv=input_param[6:9],
    #x_predict = X_input
)

tinygp_log =  _loss_tinygp(theta_dict)
print(tinygp_log)


#gp = _build_tinygp_multidimensional(theta_dict)
#_, cond_gp = gp.condition(theta_dict['y'], theta_dict['x_predict'])
#
#mu = cond_gp.mean
#std = np.sqrt(cond_gp.variance)
#mu_full = cond_gp.loc # or cond_gp.mean?

750.6751087298841
963.3958376906472


In [90]:
'''
def model_logchi2(self):
        env = 1.0 / (self.e ** 2.0 + self.jitter ** 2.0)

        #chi2 = -0.5 * (self.n * np.log(2 * np.pi) +
        #               np.sum(self.residuals ** 2 * env - np.log(env)))
        #print('{0:25s} {1:12f} {2:12f} \n'.format(self.name_ref, chi2, np.std(self.residuals)))

        return -0.5 * (self.n * np.log(2 * np.pi) +
                       np.sum(self.residuals ** 2 * env - np.log(env)))
'''